<a href="https://colab.research.google.com/github/kombidonja/be_Bff_Chat/blob/main/be_bff_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip cache purge
!pip uninstall google-generativeai

In [115]:
# Instalando o SDK do Google
# Instalando e baixando bibliotecas silenciosamente
!pip install -q -U google-generativeai > /dev/null 2>&1
!pip install -q textblob spacy > /dev/null 2>&1
!python -m spacy download pt_core_news_sm > /dev/null 2>&1

# Importações
import datetime
import random
import google.generativeai as genai
from textblob import TextBlob
import spacy


y


In [116]:
api_key = "SUA_CHAVE_API"  # Insira sua chave API do Google Generative AI
genai.configure(api_key=api_key)

generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}

safety_settings = {
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL': 'BLOCK_NONE',
    'DANGEROUS': 'BLOCK_NONE'
}

# Crie um objeto GenerativeModel
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                generation_config=generation_config,
                                safety_settings=safety_settings)

prompt = "Responda a '{texto_usuario}' de forma {tom_de_voz} considerando uma personalidade {personalidade} e estilo de linguagem {estilo_linguagem}."


In [ ]:
memoria_usuario = {}  # Dicionário para armazenar informações do usuário


def escolher_personalidade():
    """
    Permite ao usuário escolher a personalidade, tom de voz,
    estilo de linguagem e concordância do chatbot.
    """
    print("Escolha a personalidade do seu amigo virtual:")
    print("1. Extrovertido(a) e Brincalhão(ã)")
    print("2. Calmo(a) e Reflexivo(a)")
    print("3. Sábio(a) e Misterioso(a)")
    while True:
        escolha = input("Digite o número da opção desejada: ")
        if escolha in ["1", "2", "3"]:
            break
        else:
            print("Opção inválida. Tente novamente.")
    if escolha == "1":
        personalidade = "extrovertido"
        tom_de_voz = "informal"
    elif escolha == "2":
        personalidade = "calmo"
        tom_de_voz = "formal"
    else:
        personalidade = "sábio"
        tom_de_voz = "formal"
    print("Escolha o estilo de linguagem:")
    print("1. Masculino")
    print("2. Feminino")
    print("3. LGBTQIA+")
    while True:
        escolha_estilo = input("Digite o número da opção desejada: ")
        if escolha_estilo in ["1", "2", "3"]:
            break
        else:
            print("Opção inválida. Tente novamente.")
    if escolha_estilo == "1":
        estilo_linguagem = "masculino"
    elif escolha_estilo == "2":
        estilo_linguagem = "feminino"
    else:
        estilo_linguagem = "lgbtqia+"
    print("Escolha o tipo de concordância:")
    print("1. Concorda com tudo")
    print("2. Concorda baseado em informações da internet")
    print("3. Mixto (concorda com algumas coisas e discorda de outras)")
    while True:
        escolha_concordancia = input("Digite o número da opção desejada: ")
        if escolha_concordancia in ["1", "2", "3"]:
            break
        else:
            print("Opção inválida. Tente novamente.")
    if escolha_concordancia == "1":
        concordancia = "concorda_tudo"
    elif escolha_concordancia == "2":
        concordancia = "concorda_internet"
    else:
        concordancia = "mixto"
    print(f"Você escolheu um amigo(a) {personalidade} com um tom de voz {tom_de_voz}, estilo de linguagem {estilo_linguagem} e que {concordancia} com você.")
    return personalidade, tom_de_voz, estilo_linguagem, concordancia




def conversar(texto_usuario, personalidade, tom_de_voz, estilo_linguagem, concordancia, nome_usuario=None):
    """
    Processa a entrada do usuário e gera uma resposta com base na personalidade.
    """
    resposta = ""  # Inicialize a variável 'resposta' como uma string vazia



    # 1. Escuta ativa: Reconhecer emoções
    emocao = detectar_emocao(texto_usuario)  # Chame a função e atribua o resultado à variável 'emocao'
    # 2. Responder de acordo com a emoção e personalidade
    if emocao == "tristeza":
        if personalidade == "extrovertido":
            resposta = "Ei, não fica assim! Quer me contar o que aconteceu?"
        elif personalidade == "calmo":
            resposta = "Sinto muito que você esteja se sentindo triste. Deseja conversar sobre isso?"
        else:  # sábio
            resposta = "A tristeza faz parte da vida, mas lembre-se de que ela é passageira."
    # ... (adicionar respostas para outras emoções e personalidades) ...

    # 4. Lembrar detalhes: Armazenar informações
    if "nome" in texto_usuario.lower():
        nome = extrair_nome(texto_usuario)
        memoria_usuario["nome"] = nome
        resposta += f" Prazer em te conhecer, {nome}!"
    # 5. Concordância
    if concordancia == "concorda_tudo":
        resposta += " Concordo plenamente."
    elif concordancia == "concorda_internet":
        # Aqui você deve adicionar a lógica para verificar informações na internet
        # e concordar/discordar com o usuário.
        # Certifique-se de que este código esteja indentado!
        pass  # Remova este 'pass' quando implementar a lógica
    # ... (lógica para concordância mista) ...
    # Adicionar o nome do usuário à resposta (se fornecido)
    if nome_usuario:
        resposta = f"{nome_usuario}, {resposta}"
    return resposta


def detectar_emocao(texto):
    sentimento = TextBlob(texto).sentiment.polarity
    if sentimento < 0:
        return "tristeza"
    elif sentimento > 0:
        return "alegria"
    else:
        return "neutro"


def extrair_nome(texto):
    nlp = spacy.load("pt_core_news_sm")
    doc = nlp(texto)
    for entidade in doc.ents:
        if entidade.label_ == "PER":
            return entidade.text
    return None


def interagir(personalidade, nome_usuario):
    if personalidade == "extrovertido":
        interacoes_possiveis = [
            "E aí, o que está rolando?",
            "Me conta uma novidade!",
            "Qual é a boa de hoje?",
            "Estou animado para conversar! O que você quer falar?"
        ]
    elif personalidade == "calmo":
        interacoes_possiveis = [
            "Como você está se sentindo hoje?",
            "O que está passando pela sua cabeça?",
            "Quer conversar sobre algo específico?",
            "Estou aqui para ouvir."
        ]
    else:  # sábio
        interacoes_possiveis = [
            "Qual é a questão que te aflige?",
            "Compartilhe seus pensamentos comigo.",
            "A sabedoria vem da reflexão. O que você está ponderando?",
            "Estou aqui para oferecer minha perspectiva."
        ]
    interacao = random.choice(interacoes_possiveis)
    print(f"Be: {interacao}, {nome_usuario}?")


def obter_saudacao_gemini(horario, personalidade, tom_de_voz, estilo_linguagem):
    """
    Gera uma saudação aleatória usando o modelo Gemini
    com base nos parâmetros.
    """
    prompts = {
        "manha": {
            "extrovertido": [
                f"Crie uma saudação matinal alegre e {tom_de_voz} no estilo {estilo_linguagem}.",
                f"Imagine que você é um amigo virtual {personalidade}. Como você cumprimentaria alguém de manhã?"
            ],
            "calmo": [
                f"Crie uma saudação matinal calma e {tom_de_voz} no estilo {estilo_linguagem}.",
                f"Imagine que você é um amigo virtual {personalidade}. Como você cumprimentaria alguém de manhã?"
            ],
            "sábio": [
                f"Crie uma saudação matinal sãbia e {tom_de_voz} no estilo {estilo_linguagem}.",
                f"Imagine que você é um amigo virtual {personalidade}. Como você cumprimentaria alguém de manhã?"
            ]
        },
        "tarde": {
            "extrovertido": [
                f"Crie uma saudação para a tarde animada e {tom_de_voz} no estilo {estilo_linguagem}.",
                f"Imagine que você é um amigo virtual {personalidade}. Como você cumprimentaria alguém à tarde?"
            ],
            "calmo": [
                f"Crie uma saudação para a tarde calma e {tom_de_voz} no estilo {estilo_linguagem}.",
                f"Imagine que você é um amigo virtual {personalidade}. Como você cumprimentaria alguém à tarde?"
            ],
            "sábio": [
                f"Crie uma saudação para a tarde sãbia e {tom_de_voz} no estilo {estilo_linguagem}.",
                f"Imagine que você é um amigo virtual {personalidade}. Como você cumprimentaria alguém à tarde?"
            ]
        },
        "noite": {
            "extrovertido": [
                f"Crie uma saudação noturna animada e {tom_de_voz} no estilo {estilo_linguagem}.",
                f"Imagine que você é um amigo virtual {personalidade}. Como você cumprimentaria alguém à noite?"
            ],
            "calmo": [
                f"Crie uma saudação noturna calma e {tom_de_voz} no estilo {estilo_linguagem}.",
                f"Imagine que você é um amigo virtual {personalidade}. Como você cumprimentaria alguém à noite?"
            ],
            "sábio": [
                f"Crie uma saudação noturna sãbia e {tom_de_voz} no estilo {estilo_linguagem}.",
                f"Imagine que você é um amigo virtual {personalidade}. Como você cumprimentaria alguém à noite?"
            ]
        }
    }
    if 6 <= horario < 12:
        prompt = random.choice(prompts["manha"][personalidade])
    elif 12 <= horario < 18:
        prompt = random.choice(prompts["tarde"][personalidade])
    else:
        prompt = random.choice(prompts["noite"][personalidade])
    resposta = model.generate(prompt=prompt)
    return resposta.result


def iniciar_conversa():
    """
    """
    print("Olá! Eu sou o Be, seu amigo/a/x virtual.")
    nome_usuario = input("Primeiro, me diga seu nome: ")
    print(f"Prazer em te conhecer, {nome_usuario}!")
    print("Agora, me conte um pouco sobre como você gostaria que eu fosse:")
    personalidade, tom_de_voz, estilo_linguagem, concordancia = escolher_personalidade()
    print("Legal! Estou animado para conversar com você.")

    return nome_usuario, personalidade, tom_de_voz, estilo_linguagem, concordancia


# Loop principal
nome_usuario, personalidade, tom_de_voz, estilo_linguagem, concordancia = iniciar_conversa()
while True:
    interagir(personalidade, nome_usuario)  # Passando o nome para a função interagir()
    texto_usuario = input(f"{nome_usuario}: ")  # Exibindo o nome do usuário antes do input
    resposta_be = conversar(texto_usuario, personalidade, tom_de_voz, estilo_linguagem, concordancia, nome_usuario)  # Passando o nome para a função conversar()
print(f"Be: {resposta_be}")